In [1]:
import cv2
import numpy as np
from sklearn.svm import SVC
import os
import sys
import time


In [6]:
def crearCeldas(image, N):
    height, width = image.shape[:2]

  # Calculamos el número de bloques en filas y columnas
    num_blocks_rows = height // N
    num_blocks_cols = width // N

    # Dividimos la imagen en bloques de 16 filas
    blocks = np.vsplit(image, num_blocks_rows)

    # Creamos una lista para guardar todas las celdas
    celdas = []

    # Iteramos sobre los bloques
    for i, block in enumerate(blocks):
        # Dividimos cada bloque en bloques d
        # e 16 columnas
        cells = np.hsplit(block, num_blocks_cols)

        # Iteramos sobre las celdas
        for j, cell in enumerate(cells):
            # Añadimos cada celda a la lista
            celdas.append(cell)
            

    celdas = np.array(celdas)
    return celdas


In [21]:
def im_to_col(imagen, m, n):
    filas, columnas = imagen.shape
    a = m//2
    b = n//2
    imagen2 = np.zeros((m*n, filas*columnas))
    imagen_amp = cv2.copyMakeBorder(imagen, a, a, b, b, cv2.BORDER_REPLICATE)
    aux = 0
    for i in range(a, filas+a):
        for j in range(b, columnas+b):
            imagen2[:, aux] = imagen_amp[i-a:i+a+1, j-b:j+b+1].flatten()
            aux += 1
    return imagen2

In [43]:
def filtro(imagen, mascara):
    imagen2 = im_to_col(imagen, len(mascara), len(mascara[0]))
    imagen = mascara.reshape(1,-1).dot(imagen2).reshape(imagen.shape)   
    return imagen    

In [1]:
def orientaciones(phi):
    return np.abs(phi)//20

In [85]:
def feature_extraction_2(image, N, R):

    maskX = np.array([[-1, 0, 1]])
    maskY = np.array([[-1], [0], [1]])
    
    dx = filtro(image,maskX)
    dy = filtro(image,maskY)

    magnitudes = np.sqrt(dx**2+dy**2)
    PhiRad = np.arctan2(dy, dx)
    phi = np.rad2deg(PhiRad)

    orientaciones = orientaciones(phi)
    celdasMag = crearCeldas(magnitudes, N)

    celdasMask = np.zeros((9,celdasMag.shape))
    for i in range(celdasMask.shape[0]):
        celdasMask[i] = crearCeldas(orientaciones == i ,N)

    
    


    histograms = np.zeros((len(celdasMag), 9))
    for i, celda in enumerate(celdasMag):
        celdasMask*celda
        valor = np.sqrt(np.sum(histograms[i]**2)) 
        if valor != 0:
            histograms[i] = histograms[i]/valor
    vector = histograms.reshape(1,-1)
    return vector.flatten()


In [ ]:
# Cargamos la imagen
N = 16
R = 3
# Convertimos la imagen a escala de grises
image = cv2.imread('test.jpg', cv2.IMREAD_GRAYSCALE)

result = feature_extraction_2(image, N, R)

print(len(result))

In [ ]:
datasets = ["cat_dog_100"]

N = 16
R = 3

HEIGHT = 384
WIDTH = 528

for dataset in datasets:
    print("dataset: ", dataset)
    # Iniciamos el contador de tiempo
    startTimer = time.perf_counter()

    # Definimos el directorio de entrenamiento y prueba
    train_dir = dataset+"/"+"train"
    test_dir = dataset+"/"+"test"

    # Inicializamos los arrays de datos de entrenamiento y prueba
    X_train = []
    y_train = []
    X_test = []
    y_test = []

    print("Procesamos las imágenes de entrenamiento")
    # Procesamos las imágenes de entrenamiento
    for label in os.listdir(train_dir):
        label_dir = os.path.join(train_dir, label)
        print("Procesando label: ", label)
        for count, image_filename in enumerate(os.listdir(label_dir)):
            image_path = os.path.join(label_dir, image_filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            resized_image = cv2.resize(
                image, (HEIGHT, WIDTH), interpolation=cv2.INTER_LINEAR)
            features = feature_extraction_2(resized_image, N, R)
            X_train.append(features)
            y_train.append(label)

            progress = 100 * (count + 1) / len(os.listdir(label_dir))
            sys.stdout.write(
                "\r Procesando Xtrain ... (" + str(int(progress)) + " %)")
        print("")

    print("Procesamos las imágenes de prueba")
    # Procesamos las imágenes de prueba
    for label in os.listdir(test_dir):
        label_dir = os.path.join(test_dir, label)
        print("Procesando label: ", label)
        for count, image_filename in enumerate(os.listdir(label_dir)):
            image_path = os.path.join(label_dir, image_filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            resized_image = cv2.resize(
                image, (HEIGHT, WIDTH), interpolation=cv2.INTER_LINEAR)
            features = feature_extraction_2(resized_image, N, R)
            X_test.append(features)
            y_test.append(label)

            progress = 100 * (count + 1) / len(os.listdir(label_dir))
            sys.stdout.write(
                "\r Procesando Xtest ... (" + str(int(progress)) + " %)")

    # Convertimos los datos de entrenamiento y prueba a arrays de NumPy
    X_train = np.array(X_train, np.uint8)
    y_train = np.array(y_train)
    X_test = np.array(X_test, np.uint8)
    y_test = np.array(y_test)    


    # Creamos un clasificador SVM
    clf = SVC(gamma='auto')

    # Entrenamos el clasificador con los datos de entrenamiento
    clf.fit(X_train, y_train)

    # Realizamos predicciones con el clasificador entrenado
    y_pred = clf.predict(X_test)

    # Calculamos el porcentaje de acierto
    accuracy = np.mean(y_pred == y_test)
    print("Precisión: {:.2f}%".format(accuracy * 100))

    # Detenemos el contador de tiempo
    endTimer = time.perf_counter()

    # Calculamos el tiempo transcurrido
    elapsed = endTimer - startTimer
    # Obtenemos el tiempo transcurrido en minutos y segundos
    minutes, seconds = divmod(elapsed, 60)

    # Mostramos el tiempo transcurrido en minutos y segundos
    print(f"Tiempo transcurrido: {minutes:.0f} minutos {seconds:.2f} segundos")


    
